In [1]:
#Actors and Directors Who Cooperated At Least Three Times

#SQL
# Write your MySQL query statement below
Select actor_id, director_id
FROM ActorDirector
Group By actor_id, director_id
Having count(*) >=3;
#Pandas
import pandas as pd

def actors_and_directors(actor_director: pd.DataFrame) -> pd.DataFrame:
    import pandas as pd

def actors_and_directors(actor_director: pd.DataFrame) -> pd.DataFrame:
    result = (
        actor_director
        .groupby(['actor_id', 'director_id'])
        .filter(lambda x: len(x) >= 3)[['actor_id', 'director_id']]
        .drop_duplicates()
        .reset_index(drop=True)
    )
    return result

#Fix Names in a Table

#SQL
# Write your MySQL query statement below
SELECT user_id, CONCAT(UPPER(LEFT(name,1)), LOWER(SUBSTRING(name,2))) AS name
FROM Users;
#Pandas
import pandas as pd

def fix_names(users: pd.DataFrame) -> pd.DataFrame:
    users['name'] = users['name'].str[0].str.upper() + users['name'].str[1:].str.lower()
    return users[['user_id', 'name']]

#Combine Two Tables

#SQL
# Write your MySQL query statement below
Select
    p.firstName,
    p.lastName,
    a.city,
    a.state
From Person p
Left Join Address a
On p.personId=a.personId

#Pandas
import pandas as pd

def combine_two_tables(person: pd.DataFrame, address: pd.DataFrame) -> pd.DataFrame:
    result = (
        person
        .merge(address, on='personId', how='left')
        [['firstName', 'lastName', 'city', 'state']]
    )
    return result

#Replace Employee ID With The Unique Identifier


#SQL
# Write your MySQL query statement below
SELECT
    IFNULL(u.unique_id, null) AS unique_id,
    e.name
FROM Employees e
Left JOIN EmployeeUNI u ON e.id = u.id;
#Pandas
import pandas as pd

def replace_employee_id(employees: pd.DataFrame, employee_uni: pd.DataFrame) -> pd.DataFrame:
    result = (
        employees
        .merge(employee_uni, on='id', how='left')
        [['unique_id', 'name']]
    )
    return result
#Second Highest Salary

#SQL
# Write your MySQL query statement below
SELECT
    IFNULL(
        (SELECT salary
         FROM Employee
         ORDER BY salary DESC
         LIMIT 1 OFFSET 1),
        NULL
    ) AS SecondHighestSalary;
#Pandas
import pandas as pd

def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    sorted_salaries = employee['salary'].drop_duplicates().sort_values(ascending=False).reset_index(drop=True)

    second_highest = sorted_salaries[1] if len(sorted_salaries) > 1 else None

    return pd.DataFrame({'SecondHighestSalary': [second_highest]})
#Department Top Three Salaries

#SQL
# Write your MySQL query statement below
SELECT
    d.name AS Department,
    e.name AS Employee,
    e.salary AS Salary
FROM Employee e
LEFT JOIN Department d ON e.departmentId = d.id
WHERE (
    SELECT COUNT(DISTINCT e2.salary)
    FROM Employee e2
    WHERE e2.departmentId = e.departmentId
      AND e2.salary > e.salary
) < 3
ORDER BY d.name, e.salary DESC, e.name ASC;
#Pandas
import pandas as pd

def top_three_salaries(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    df = employee.merge(department, left_on='departmentId', right_on='id', how='left', suffixes=('_emp', '_dept'))

    df['rank'] = df.groupby('departmentId')['salary'].rank(method='dense',  ascending=False)

    df_top3 = df[df['rank'] <= 3]

    result = df_top3[['name_dept', 'name_emp', 'salary']].rename(
        columns={'name_dept': 'Department', 'name_emp': 'Employee', 'salary': 'Salary'}
    )

    result = result.sort_values(by=['Department', 'Salary', 'Employee'], ascending=[True, False, True]).reset_index(drop=True)

    return result

#List the Products Ordered in a Period

#SQL
# Write your MySQL query statement below
SELECT
    p.product_name,
    SUM(o.unit) as unit
FROM Orders o
JOIN Products p ON o.product_id = p.product_id
WHERE o.order_date >= '2020-02-01'
  AND o.order_date < '2020-03-01'
GROUP BY p.product_name
HAVING SUM(o.unit) >= 100;
#Pandas
import pandas as pd

def list_products(products: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    df = orders.merge(products, on='product_id', how='inner')
    df_filtered = df[(df['order_date'] >= '2020-02-01') & (df['order_date'] < '2020-03-01')]
    df_grouped = df_filtered.groupby('product_name', as_index=False)['unit'].sum()
    result = df_grouped[df_grouped['unit'] >= 100]
    return result
#Project Employees I

#SQL
# Write your MySQL query statement below
Select
    p.project_id,
    sum(e.experience_years)/count(*) as average_years
From Project p
Left Join Employee e on e.employee_id=p.employee_id
Group by p.project_id;

#Pandas
import pandas as pd

def project_employees_i(project: pd.DataFrame, employee: pd.DataFrame) -> pd.DataFrame:
    df = project.merge(employee, left_on='employee_id', right_on='employee_id', how='left')
    result = df.groupby('project_id', as_index=False).agg(average_years=('experience_years', lambda x: x.sum() / x.count() if x.count() > 0 else 0))
    return result

# Game Play Analysis IV

#SQL
# Write your MySQL query statement below
SELECT
    ROUND(
        COUNT(DISTINCT next_day.player_id) / COUNT(DISTINCT first_day.player_id),
        2
    ) AS fraction
FROM
    (SELECT player_id, MIN(event_date) AS first_login
     FROM Activity
     GROUP BY player_id) AS first_day
LEFT JOIN Activity AS next_day
    ON next_day.player_id = first_day.player_id
   AND next_day.event_date = DATE_ADD(first_day.first_login, INTERVAL 1 DAY);
#Pandas
import pandas as pd

def gameplay_analysis(activity: pd.DataFrame) -> pd.DataFrame:
    first_day = activity.groupby('player_id', as_index=False)['event_date'].min().rename(columns={'event_date': 'first_login'})
    next_day = first_day.merge(
        activity,
        left_on=['player_id'],
        right_on=['player_id'],
        how='left'
    )
    next_day['event_date'] = pd.to_datetime(next_day['event_date'])
    next_day['first_login'] = pd.to_datetime(next_day['first_login'])
    next_day_filtered = next_day[next_day['event_date'] == next_day['first_login'] + pd.Timedelta(days=1)]
    players_first_day = first_day['player_id'].nunique()
    players_next_day = next_day_filtered['player_id'].nunique()
    fraction = round(players_next_day / players_first_day, 2) if players_first_day > 0 else 0.0
    return pd.DataFrame({'fraction': [fraction]})

